# Fun with Neural Nets

Let's build a neural network to recognize handwritten digits using [Kaggle's Digit Recognizer](https://www.kaggle.com/c/digit-recognizer) data!

From Kaggle:

> MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

![handwritten digits](https://imgur.com/0Qr0ATx.jpg)

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import utils

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

### Loading our training data

Load `train.csv` from Kaggle into a pandas DataFrame.

In [93]:
train = pd.read_csv('train.csv')

In [94]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Setting up X and y

NOTE: Keras requires a `numpy` matrix, it doesn't work with `pandas`.

In [95]:
X = train[train.columns[1:]].values
y = train['label']

In [96]:
X = X.astype('float32')

### Preprocessing

1. When dealing with image data, you need to normalize your `X` by dividing each value by the max number of pixels (255).
2. Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix

In [97]:
X = X/255
y = utils.to_categorical(y)

In [98]:
X.shape

(42000, 784)

In [99]:
X[0].shape

(784,)

### Train/Test Split

We want to create a validation set that the model will never see to approximate how it's going to do with Kaggle's `test.csv`.

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [101]:
X_train[0].shape

(784,)

### Creating our neural network

In [106]:
model = Sequential()
model.add(Flatten(input_shape= (784, 1)))
model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dropout(.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

### Compiling our model

Since this is a multiclass classification problem, our loss function is `categorical_crossentropy`.

In [107]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

### Fitting our model

In [109]:
model.fit(X_train, y_train,  batch_size=256, validation_data=(X_test, y_test), epochs=10, verbose=1)

Epoch 1/10
124/124 [==============================] - 2s 18ms/step - loss: 0.0615 - accuracy: 0.9817 - val_loss: 0.0753 - val_accuracy: 0.9778
Epoch 2/10
124/124 [==============================] - 2s 15ms/step - loss: 0.0557 - accuracy: 0.9830 - val_loss: 0.0750 - val_accuracy: 0.9784
Epoch 3/10
124/124 [==============================] - 2s 14ms/step - loss: 0.0468 - accuracy: 0.9861 - val_loss: 0.0750 - val_accuracy: 0.9774
Epoch 4/10
124/124 [==============================] - 2s 14ms/step - loss: 0.0447 - accuracy: 0.9870 - val_loss: 0.0712 - val_accuracy: 0.9787
Epoch 5/10
124/124 [==============================] - 2s 15ms/step - loss: 0.0398 - accuracy: 0.9881 - val_loss: 0.0725 - val_accuracy: 0.9784
Epoch 6/10
124/124 [==============================] - 2s 15ms/step - loss: 0.0366 - accuracy: 0.9894 - val_loss: 0.0716 - val_accuracy: 0.9789
Epoch 7/10
124/124 [==============================] - 2s 14ms/step - loss: 0.0354 - accuracy: 0.9907 - val_loss: 0.0694 - val_accuracy: 0.9790

### Loading in Kaggle's `test.csv`

In [6]:
test = pd.read_csv('test.csv')
test = test / 255

In [121]:
predict_x=model.predict(test) 
classes_x=np.argmax(predict_x,axis=1)

In [122]:
test['Label'] = classes_x

### Preparing our Kaggle submission

1. Adding our predictions to a column called `Label`
2. We'll need to manually create the `ImageId` column.

In [7]:
test['ImageId'] = range(1,test.shape[0] + 1)

### Creating our submission csv

Remember to set `index=False`!

In [124]:
test[['ImageId', 'Label']].to_csv('submission.csv', index=False)